# Introduction

<a target="_blank" href="https://colab.research.google.com/github/shahules786/openai-cookbook/blob/ragas/examples/evaluation/ragas/openai-ragas-eval-cookbook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Ragas is the de-facto opensource standard for RAG evaluations. Ragas provides features and methods to help evaluate RAG applications. In this notebook we will cover basic steps for evaluating your RAG application with Ragas. 

### Contents
- [Prerequisites]()
- [Dataset preparation]()
- [Evaluation]()
- [Analysis]()

### Prerequisites
- Ragas is a python package and we can install it using pip
- Some documents to build our simple RAG pipeline
- Ragas uses model guided techniques underneath to produce scores for each metric. In this tutorial, we will use OpenAI `gpt-3.5-turbo` and `text-embedding-ada-002`. These are the default models used in ragas but you can use any LLM or Embedding of your choice by referring to this [guide](https://docs.ragas.io/en/stable/howtos/customisations/bring-your-own-llm-or-embs.html). I highly recommend that you try this notebook with open-ai so that you get a feel of it with ease.

In [ ]:
! pip install -q ragas llama-index

In [ ]:
!git clone https://huggingface.co/datasets/explodinggradients/prompt-engineering-guide-papers

In [16]:
import os
os.environ["OPENAI_API_KEY"] = "<your-openai-key>"

try:
  import google.colab
  PATH = "/content/prompt-engineering-guide-papers""
except:
  PATH = "./prompt-engineering-guide-papers"

And that's it. You're ready to go.

## Dataset preparation

Evaluating any ML pipeline will require several data points that constitues a test dataset. For Ragas, the data points required for evaluating your RAG completely are

- `question`: A question or query that is relevant to your RAG.
- `contexts`: The retrieved contexts corresponding to each question. This is a `list[list]` since each question can retrieve multiple text chunks.
- `answer`:  The answer generated by your RAG corresponding to each question.
- `ground_truth`: The expected correct answer corresponding to each question.

For the purpose of this notebook, I have this dataset prepared from a simple RAG that I created myself to help me with NLP research. Let's use it.

In [3]:
from datasets import load_dataset

In [21]:
eval_dataset = load_dataset("explodinggradients/prompt-engineering-guide-papers")
eval_dataset = eval_dataset['test'].to_pandas()
eval_dataset.head()

Found cached dataset json (/Users/shahules/.cache/huggingface/datasets/explodinggradients___json/explodinggradients--prompt-engineering-guide-papers-9147f70034f5334d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

,question,ground_truth
0,How does instruction tuning affect the zero-sh...,For larger models on the order of 100B paramet...
1,What is the Zero-shot-CoT method and how does ...,Zero-shot-CoT is a zero-shot template-based pr...
2,How does prompt tuning affect model performanc...,Prompt tuning improves model performance in im...
3,What is the purpose of instruction tuning in l...,The purpose of instruction tuning in language ...
4,What distinguishes Zero-shot-CoT from Few-shot...,Zero-shot-CoT differs from Few-shot-CoT in tha...


As you can see, the dataset contains two of the required attributes mentioned,that is `question` and `ground_truth` answers. Now we can move on our next step to collect the other two attributes.

**Note:**
*We know that it's hard to formulate a test data containing Question and ground truth answer pairs when starting out. We have the perfect solution for this in this form of a ragas synthetic test data generation feature. The questions and ground truth answers were created by [ragas synthetic data generation](https://colab.research.google.com/github/shahules786/openai-cookbook/blob/ragas/examples/evaluation/ragas/openai-ragas-synthetic-test.ipynb) feature. Check it out here once you finish this notebook*

#### Simple RAG pipeline

Now with the above step we have two attributes needed for evaluation, that is `question` and `ground_truth` answers. We now need to feed these test questions to our RAG pipeline to collect the other two attributes, ie `contexts` and `answer`.  Let's build a simple RAG using llama-index to do that. 

In [5]:
import nest_asyncio
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.service_context import ServiceContext
from datasets import Dataset

nest_asyncio.apply()


def build_query_engine(documents):
    vector_index = VectorStoreIndex.from_documents(
        documents, service_context=ServiceContext.from_defaults(chunk_size=512),
    )

    query_engine = vector_index.as_query_engine(similarity_top_k=3)
    return query_engine

# Function to evaluate as Llama index does not support async evaluation for HFInference API
def generate_responses(query_engine, test_questions, test_answers):
  responses = [query_engine.query(q) for q in test_questions]

  answers = []
  contexts = []
  for r in responses:
    answers.append(r.response)
    contexts.append([c.node.get_content() for c in r.source_nodes])
  dataset_dict = {
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
  }
  if test_answers is not None:
    dataset_dict["ground_truth"] = test_answers
  ds = Dataset.from_dict(dataset_dict)
  return ds

In [7]:
reader = SimpleDirectoryReader(PATH,num_files_limit=30, required_exts=[".pdf"])
documents = reader.load_data()


In [27]:
test_questions = eval_dataset['question'].values.tolist()
test_answers = eval_dataset['ground_truth'].values.tolist()

In [28]:
query_engine1 = build_query_engine(documents)
result_ds = generate_responses(query_engine1, test_questions, test_answers)

## Evaluation
For evaluation ragas provides several metrics which is aimed to quantify the end-end performance of the pipeline and also the component wise performance of the pipeline. For this tutorial let's consider few of them

**Note**: *Refer to our [metrics](https://docs.ragas.io/en/stable/concepts/metrics/index.html) docs to read more about different metrics.*

In [29]:
from ragas.metrics import answer_correctness, faithfulness 
from ragas import evaluate

ragas_results = evaluate(result_ds, metrics=[answer_correctness, faithfulness ])

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

## Analysis
You can export the individual scores to dataframe and analyse it. You can also add [callbacks and tracing](https://docs.ragas.io/en/latest/howtos/applications/tracing.html) to ragas to do indepth analysis.

In [34]:
ragas_results.to_pandas().head(5)

,question,answer,contexts,ground_truth,answer_correctness,faithfulness
0,How does instruction tuning affect the zero-sh...,Instruction tuning enhances the zero-shot perf...,[34\nthe effectiveness of different constructi...,For larger models on the order of 100B paramet...,0.781983,1.0
1,What is the Zero-shot-CoT method and how does ...,Zero-shot-CoT is a method that involves append...,[Plan-and-Solve Prompting: Improving Zero-Shot...,Zero-shot-CoT is a zero-shot template-based pr...,0.667026,1.0
2,How does prompt tuning affect model performanc...,Prompt tuning can impact model performance in ...,[4 C. Liu et al.\nto generate results directly...,Prompt tuning improves model performance in im...,0.396040,1.0
3,What is the purpose of instruction tuning in l...,The purpose of instruction tuning in language ...,"[In practice,\ninstruction tuning offers a gen...",The purpose of instruction tuning in language ...,0.694074,1.0
4,What distinguishes Zero-shot-CoT from Few-shot...,Zero-shot-CoT conditions the LM on a single pr...,[Wei et al. (2022b ) observe that the success ...,Zero-shot-CoT differs from Few-shot-CoT in tha...,0.530018,1.0


**If you liked this tutorial, checkout [ragas](https://github.com/explodinggradients/ragas) and consider leaving a star!**